In [ ]:
from pyspark.sql import SparkSession
# Import the SparkSession class from the pyspark.sql module
spark = SparkSession.builder.master("local[*]").appName("Test SQL app").getOrCreate()
# .master("local[*]") \  - Set the Spark master URL to run locally using all available CPU cores.
 # .appName("Test SQL app") \ - Set the name of the Spark application.
 # .getOrCreate() - Get an existing SparkSession or create a new one if it doesn't exist.

In [ ]:
df=spark.read.format("jdbc").options(driver="com.mysql.cj.jdbc.Driver",\
                                     user="root",\
                                     password="password",\
                                     url="jdbc:mysql://localhost:3306/classicmodels",\
                                     dbtable="classicmodels.orders").load()

# Read data from a MySQL database into a Spark DataFrame using JDBC
# df = spark.read \
#     .format("jdbc") \                                  # Specify the format as JDBC to read from a relational database
#     .options(                                          # Provide connection options:
#        driver="com.mysql.cj.jdbc.Driver",              # - The JDBC driver class for MySQL
#        user="root",                                    # - Username for connecting to the MySQL database
#        password="password",                            # - Password for the MySQL user
#        url="jdbc:mysql://localhost:3306/classicmodels",# - JDBC URL including host, port, and database name
#        dbtable="classicmodels.orders"                  # - Table to read (schema.table or just table name if default schema)
#     ).load()                                           # Load the table data into a Spark DataFrame


In [3]:
df.count()  # return number of rows

326

In [4]:
df.printSchema()  # return schema

root
 |-- orderNumber: integer (nullable = true)
 |-- orderDate: date (nullable = true)
 |-- requiredDate: date (nullable = true)
 |-- shippedDate: date (nullable = true)
 |-- status: string (nullable = true)
 |-- comments: string (nullable = true)
 |-- customerNumber: integer (nullable = true)



In [5]:
df.show()  # return all columns with data

+-----------+----------+------------+-----------+-------+--------------------+--------------+
|orderNumber| orderDate|requiredDate|shippedDate| status|            comments|customerNumber|
+-----------+----------+------------+-----------+-------+--------------------+--------------+
|      10100|2003-01-06|  2003-01-13| 2003-01-10|Shipped|                NULL|           363|
|      10101|2003-01-09|  2003-01-18| 2003-01-11|Shipped|Check on availabi...|           128|
|      10102|2003-01-10|  2003-01-18| 2003-01-14|Shipped|                NULL|           181|
|      10103|2003-01-29|  2003-02-07| 2003-02-02|Shipped|                NULL|           121|
|      10104|2003-01-31|  2003-02-09| 2003-02-01|Shipped|                NULL|           141|
|      10105|2003-02-11|  2003-02-21| 2003-02-12|Shipped|                NULL|           145|
|      10106|2003-02-17|  2003-02-24| 2003-02-21|Shipped|                NULL|           278|
|      10107|2003-02-24|  2003-03-03| 2003-02-26|Shipped|Dif

Example 2: 

Read with a custom query:

In [ ]:
query="(select * from orders where customerNumber = 144) as cust"
# Define a SQL subquery that filters the 'orders' table for customerNumber = 144
# The subquery is wrapped in parentheses and given an alias ('cust'), which is required by JDBC
df=spark.read.format("jdbc").options(driver="com.mysql.cj.jdbc.Driver",\
                                     user="root",\
                                     password="password",\

url="jdbc:mysql://localhost:3306/classicmodels",\
                                     dbtable=query).load()

df.show()

+-----------+----------+------------+-----------+-------+--------------------+--------------+
|orderNumber| orderDate|requiredDate|shippedDate| status|            comments|customerNumber|
+-----------+----------+------------+-----------+-------+--------------------+--------------+
|      10112|2003-03-24|  2003-04-03| 2003-03-29|Shipped|Customer requeste...|           144|
|      10320|2004-11-03|  2004-11-13| 2004-11-07|Shipped|                NULL|           144|
|      10326|2004-11-09|  2004-11-16| 2004-11-10|Shipped|                NULL|           144|
|      10334|2004-11-19|  2004-11-28|       NULL|On Hold|The outstaniding ...|           144|
+-----------+----------+------------+-----------+-------+--------------------+--------------+



Example: 3:

Querying Multiple Records from MySQL in PySpark:

In [11]:
query="(select * from orders where customerNumber = 144 or customerNumber = 128) as cust"

df=spark.read.format("jdbc").options(driver="com.mysql.cj.jdbc.Driver",\
                                     user="root",\
                                     password="password",\
                                     url="jdbc:mysql://localhost:3306/classicmodels",\
                                     dbtable=query \
                                    ).load()
df.show()


+-----------+----------+------------+-----------+-------+--------------------+--------------+
|orderNumber| orderDate|requiredDate|shippedDate| status|            comments|customerNumber|
+-----------+----------+------------+-----------+-------+--------------------+--------------+
|      10101|2003-01-09|  2003-01-18| 2003-01-11|Shipped|Check on availabi...|           128|
|      10230|2004-03-15|  2004-03-24| 2004-03-20|Shipped|Customer very con...|           128|
|      10300|2003-10-04|  2003-10-13| 2003-10-09|Shipped|                NULL|           128|
|      10323|2004-11-05|  2004-11-12| 2004-11-09|Shipped|                NULL|           128|
|      10112|2003-03-24|  2003-04-03| 2003-03-29|Shipped|Customer requeste...|           144|
|      10320|2004-11-03|  2004-11-13| 2004-11-07|Shipped|                NULL|           144|
|      10326|2004-11-09|  2004-11-16| 2004-11-10|Shipped|                NULL|           144|
|      10334|2004-11-19|  2004-11-28|       NULL|On Hold|The